In [1]:
import pandas as pd
import numpy as np
from utils import *
pd.set_option('display.max_columns', None)

noaa_austin_path = '../data/Austin_NOAA.csv'
om_austin_path = '../data/Austin_OM.csv'
wrh_austin_path = '../data/Austin_WRH.csv'
air_austin_path = '../data/Austin_Air_Quality.csv'
solar_austin_path = '../data/Austin_Solar_Soil.csv'

noaa_nyc_path = '../data/NYC_NOAA.csv'
om_nyc_path = '../data/NYC_OM.csv'
wrh_nyc_path = '../data/NYC_WRH.csv'
air_nyc_path = '../data/NYC_Air_Quality.csv'
solar_nyc_path = '../data/NYC_Solar_Soil.csv'

noaa_miami_path = '../data/Miami_NOAA.csv'
om_miami_path = '../data/Miami_OM.csv'
wrh_miami_path = '../data/Miami_WRH.csv'
air_miami_path = '../data/Miami_Air_Quality.csv'
solar_miami_path = '../data/Miami_Solar_Soil.csv'

noaa_chicago_path = '../data/Chicago_NOAA.csv'
om_chicago_path = '../data/Chicago_OM.csv'
wrh_chicago_path = '../data/Chicago_WRH.csv'
air_chicago_path = '../data/Chicago_Air_Quality.csv'
solar_chicago_path = '../data/Chicago_Solar_Soil.csv'

daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_austin_path, om_austin_path, solar_austin_path, wrh_austin_path, air_austin_path)
#daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_nyc_path, om_nyc_path, solar_nyc_path, wrh_nyc_path, air_nyc_path)
#daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_miami_path, om_miami_path, solar_miami_path, wrh_miami_path, air_miami_path)
#daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_chicago_path, om_chicago_path, solar_chicago_path, wrh_chicago_path, air_chicago_path)

c:\Users\kmfoc\My Drive\BU Files\BU Masters\Work\Spring 2024\CS542\Common Task\organized\models\utils.py:79: DtypeWarning: Columns (13,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

columns_to_ignore = ['date', 'next_day_max_temp']
target_column = 'next_day_max_temp'

data = all_df

X = data.drop(columns=columns_to_ignore)
y = data[target_column]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
# XGBoost
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

In [6]:
# import explained variance score
from sklearn.metrics import explained_variance_score
import plotly.graph_objs as go

# Make predictions on the test data
y_pred = model.predict(X_test_scaled)

# Print explained variance on the test data
print("Explained Variance (R-squared) on Test Data:", explained_variance_score(y_test, y_pred))

# Print mean squared error on the test data
mse = np.mean((y_pred - y_test)**2)
print("Mean Squared Error (MSE) on Test Data:", mse)

mae = np.mean(np.abs(y_pred - y_test))
print("Mean Absolute Error (MAE) on Test Data:", mae)

std_error = np.std(y_pred - y_test)
print("Standard Error on Test Data:", std_error)

confidence_interval = 1.96 * std_error
print("95% Confidence Interval on Test Data:", confidence_interval)

# Create upper and lower bounds for uncertainty shading
upper_bound = y_pred + confidence_interval
lower_bound = y_pred - confidence_interval

# Plot the predicted vs. actual values
# Create traces for actual and predicted values
trace_actual = go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test,
    mode='lines',
    name='Actual'
)

trace_predicted = go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred,
    mode='lines',
    name='Predicted'
)

# Create traces for the uncertainty shading
trace_upper_bound = go.Scatter(
    x=np.arange(len(y_pred)),
    y=upper_bound,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    showlegend=False
)

trace_lower_bound = go.Scatter(
    x=np.arange(len(y_pred)),
    y=lower_bound,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty',
    showlegend=False
)

# Create the layout for the figure
layout = go.Layout(
    title='Predicted vs. Actual Values with 95% Confidence Interval',
    xaxis=dict(title='Data Points'),
    yaxis=dict(title='Values')
)

# Create the figure
fig = go.Figure(data=[trace_actual, trace_predicted, trace_upper_bound, trace_lower_bound], layout=layout)

# Display the figure
fig.show()

Explained Variance (R-squared) on Test Data: 0.6279741018413075
Mean Squared Error (MSE) on Test Data: 85.69016839817611
Mean Absolute Error (MAE) on Test Data: 6.49526407645672
Standard Error on Test Data: 9.168628243973169
95% Confidence Interval on Test Data: 17.97051135818741
